# <h1 align=center> **PROYECTO INDIVIDUAL Nº2**
## <h1 align=center> **KPIS**
## <h3 align=center> **Salomón Orozco Jaramillo**
---

In [1]:
# import pandas for data manipulation
import pandas as pd
# import numpy for numerical operations
import numpy as np

In [9]:
# Define a function to assign a semester based on the month of a given date
def semestre(date):
    if date.month <= 6:
        return 1  # First semester
    else:
        return 2  # Second semester

In [2]:
# Read the CSV file containing data about homicides and store it in a DataFrame named 'homicidios'
homicidios = pd.read_csv(r'datasets/homicidios.csv')
# Read the CSV file containing data about injuries and store it in a DataFrame named 'lesiones'
lesiones = pd.read_csv(r'datasets/lesiones.csv')


In [3]:
# Convert the 'fecha' column in the 'homicidios' DataFrame to datetime objects
homicidios['fecha'] = pd.to_datetime(homicidios['fecha'])
# Convert the 'fecha_y' column in the 'lesiones' DataFrame to datetime objects
lesiones['fecha_y'] = pd.to_datetime(lesiones['fecha_y'])


In [5]:
# Rename columns in the 'lesiones' DataFrame
lesiones.rename(columns={'franja_hora': 'franja_horaria', 'fecha_y': 'fecha', 'otra_direccion': 'direccion_normalizada'}, inplace=True)
# Concatenate the 'homicidios' and modified 'lesiones' DataFrames into a new DataFrame named 'siniestros'
siniestros = pd.concat([homicidios, lesiones])
# Save the 'siniestros' DataFrame to a CSV file named 'siniestros.csv' without including the index
siniestros.to_csv('datasets/siniestros.csv', index=False)
# Display information about the 'siniestros' DataFrame
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28315 entries, 0 to 27597
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_hecho               28315 non-null  object        
 1   n_victimas             28315 non-null  int64         
 2   fecha                  28315 non-null  datetime64[ns]
 3   franja_horaria         28307 non-null  float64       
 4   direccion_normalizada  7177 non-null   object        
 5   comuna                 27201 non-null  object        
 6   longitud               26659 non-null  object        
 7   latitud                26659 non-null  object        
 8   victima                15484 non-null  object        
 9   acusado                10374 non-null  object        
 10  rol                    706 non-null    object        
 11  sexo                   26081 non-null  object        
 12  edad                   23791 non-null  float64       
 13  graved

In [7]:
# Replace values in the 'gravedad' column of the 'siniestros' DataFrame
siniestros['gravedad'].replace({'GRAVE': 'grave', 'FATAL': 'fatal'}, inplace=True)

In [10]:
# Add the 'semestre' column using the previously defined function
siniestros['semestre'] = siniestros['fecha'].apply(semestre)
# Extract the year from the 'fecha' column and add it as a new column named 'year'
siniestros['year'] = siniestros['fecha'].dt.year


In [12]:
# Group by year, semester, and severity level of the accident, and count the occurrences
kpi1 = siniestros.groupby(['year', 'semestre', 'gravedad']).size().reset_index(name='cantidad_homicidios')
# Filter rows where severity level is 'fatal'
kpi1 = kpi1[kpi1['gravedad'] == 'fatal']
# Shift the 'cantidad_homicidios' column by one period to get the count of fatalities in the previous semester
kpi1['semestre_anterior'] = kpi1['cantidad_homicidios'].shift(periods=1, fill_value=0)
# Calculate the current and previous semester rates per 100,000 inhabitants
kpi1['Tasa Actual'] = (kpi1['cantidad_homicidios'] / 3121707) * 100000
kpi1['Tasa Anterior'] = (kpi1['semestre_anterior'] / 3121707) * 100000
# Calculate the percentage variation between the current and previous semester rates
kpi1['variacion%'] = (kpi1['Tasa Actual'] - kpi1['Tasa Anterior']) / kpi1['Tasa Anterior'] * 100

In [13]:
# Export csv of the kpi_1
kpi1.to_csv('datasets/kpi1.csv', index = False)

In [16]:
# Create KPI_2
# Group by year, victim, and severity level of the accident, and count the occurrences
kpi2 = siniestros.groupby(['year', 'victima', 'gravedad']).size().reset_index(name='cantidad_homicidios')
# Filter rows where severity level is 'fatal'
kpi2 = kpi2[kpi2['gravedad'] == 'fatal']
# Filter rows where victim is 'MOTO'
filtro_moto = kpi2[kpi2['victima'] == 'MOTO']
kpi2 = filtro_moto.reset_index()
kpi2.drop(columns=['index'], inplace=True)  # Remove the extra index column
# Shift the 'cantidad_homicidios' column by one period to get the count of fatalities in the previous year
kpi2['last_year'] = kpi2['cantidad_homicidios'].shift(periods=1, fill_value=0)
# Calculate the current and previous year rates per 100,000 inhabitants
kpi2['Tasa Actual'] = (kpi2['cantidad_homicidios'] / 3121707) * 100000
kpi2['Tasa Anterior'] = (kpi2['last_year'] / 3121707) * 100000
# Calculate the percentage variation between the current and previous year rates per 100,000 inhabitants 
kpi2['variacion%'] = (kpi2['Tasa Actual'] - kpi2['Tasa Anterior']) / kpi2['Tasa Anterior'] * 100

In [18]:
# Export KPI_2
kpi2.to_csv('datasets/kpi2.csv', index = False)